# Quality-of-Life Utilities - Examples

This notebook demonstrates the new QoL utilities for data engineers.

## Setup

Initialize your SQL provider and Spark session.

In [ ]:
from pyspark.sql import SparkSession
from odibi_de_v2.connector import SQLServerProvider

# Get Spark session
spark = SparkSession.getActiveSession()

# Initialize SQL provider (configure with your connection details)
sql_provider = SQLServerProvider()

# Set your project details
PROJECT = "Energy Efficiency"
ENV = "qat"

## 1. Quick Health Check

Run a comprehensive health check on your project.

In [ ]:
from odibi_de_v2.utils import quick_health_check

quick_health_check(sql_provider, spark, PROJECT, ENV)

## 2. Project Summary

Get a comprehensive overview of your project.

In [ ]:
from odibi_de_v2.utils import print_project_summary

print_project_summary(sql_provider, spark, PROJECT, ENV)

## 3. List All Projects

See all configured projects in your environment.

In [ ]:
from odibi_de_v2.utils import list_projects

projects = list_projects(sql_provider, spark, ENV)

print(f"\n📁 Projects in {ENV}:")
for i, project in enumerate(projects, 1):
    print(f"{i}. {project}")

## 4. Layer Summary

Get statistics for each layer.

In [ ]:
from odibi_de_v2.utils import get_layer_summary

layers = get_layer_summary(sql_provider, spark, PROJECT, ENV)

print("\n📊 Layer Summary:")
for layer in layers:
    print(f"{layer['layer']:15} {layer['enabled_count']:3} enabled, {layer['disabled_count']:3} disabled")

## 5. Validate Configurations

Validate all transformation configurations.

In [ ]:
from odibi_de_v2.config import ConfigValidator
from odibi_de_v2.core import DataType

# Load configs
query = f"""
SELECT *
FROM TransformationRegistry
WHERE LOWER(project) = LOWER('{PROJECT}')
AND environment = '{ENV}'
"""

df = sql_provider.read(
    data_type=DataType.SQL,
    container="",
    path_prefix="",
    object_name=query,
    spark=spark
)

configs = [row.asDict() for row in df.collect()]

# Validate
validator = ConfigValidator()
report = validator.generate_validation_report(configs)

print(f"\n✅ Valid: {report['valid']}/{report['total']} ({report['pass_rate']})")
print(f"❌ Invalid: {report['invalid']}")

if report['issues']:
    print(f"\n⚠️ Issues found:")
    for issue in report['issues'][:5]:
        print(f"\n{issue['id']}:")
        for err in issue['errors']:
            print(f"  ❌ {err}")
        for warn in issue['warnings']:
            print(f"  ⚠️  {warn}")

## 6. Check Recent Failures

Identify transformations that failed recently.

In [ ]:
from odibi_de_v2.utils import get_failed_transformations

failed = get_failed_transformations(sql_provider, spark, PROJECT, ENV, hours=24)

if failed:
    print(f"\n❌ Failed transformations (last 24h): {len(failed)}")
    for fail in failed[:5]:
        print(f"\n- {fail['transformation_id']}")
        print(f"  Time: {fail['start_time']}")
        print(f"  Error: {fail.get('error_message', 'Unknown')[:80]}")
else:
    print("\n✅ No failures in the last 24 hours")

## 7. Get Specific Configuration

Retrieve configuration for a specific transformation.

In [ ]:
from odibi_de_v2.utils import get_transformation_config
import json

# Replace with your transformation ID
TRANSFORM_ID = "your-transformation-id"

config = get_transformation_config(sql_provider, spark, TRANSFORM_ID, ENV)

if config:
    print(f"\n📋 Configuration for {TRANSFORM_ID}:")
    print(f"\nModule: {config['module']}")
    print(f"Function: {config['function']}")
    print(f"Layer: {config['layer']}")
    print(f"\nInputs: {json.dumps(config['inputs'], indent=2)}")
    print(f"\nOutputs: {json.dumps(config['outputs'], indent=2)}")
else:
    print(f"❌ Configuration not found for {TRANSFORM_ID}")

## 8. Export Project Configuration

Export entire project config to JSON for backup or review.

In [ ]:
from odibi_de_v2.utils import export_project_config
import datetime

# Export to file
today = datetime.date.today().strftime("%Y%m%d")
output_file = f"{PROJECT.replace(' ', '_')}_{ENV}_{today}.json"

config = export_project_config(sql_provider, spark, PROJECT, ENV, output_file)

print(f"\n✅ Exported {config['config_count']} configurations to {output_file}")

## 9. Detailed Health Check

Run detailed diagnostics with programmatic access to results.

In [ ]:
from odibi_de_v2.utils import HealthCheck

hc = HealthCheck(sql_provider, spark, PROJECT, ENV)
report = hc.run_full_check()

# Access specific checks
print(f"\n🏥 Health Status: {report['overall_health']}")
print(f"\nData Quality Issues: {len(report['checks'].get('data_quality', []))}")
print(f"JSON Validity Issues: {len(report['checks'].get('json_validity', []))}")
print(f"Missing Functions: {len(report['checks'].get('missing_functions', []))}")
print(f"Duplicate IDs: {len(report['checks'].get('duplicate_ids', []))}")

# Full report
hc.print_report(report)

## 10. Check Specific Layer Health

Verify health of a specific layer.

In [ ]:
from odibi_de_v2.utils import HealthCheck

hc = HealthCheck(sql_provider, spark, PROJECT, ENV)

for layer in ["Bronze", "Silver_1", "Silver_2", "Gold_1"]:
    result = hc.check_layer(layer)
    status = "✅" if result['healthy'] else "❌"
    print(f"{status} {layer}: {result['config_count']} configs")
    
    if not result['healthy']:
        for issue in result['issues'][:3]:
            print(f"   - {issue}")

## 💡 Tips

1. **Daily Routine**: Run `quick_health_check()` and `print_project_summary()` each morning
2. **Before Deployment**: Always run validation and health check
3. **Troubleshooting**: Use `get_failed_transformations()` and `get_transformation_config()` to debug
4. **Backup**: Regularly export configs with `export_project_config()`
5. **CLI**: For quick checks, use the CLI commands instead of notebooks

See `QOL_UTILITIES.md` for complete documentation.